For the SQL portion of the project, we are first importing all the necessary packages. We will be using `os`,`sqlite3`, `json`, and `pandas`. We then check we are in the correct directory using `os.getcwd()`.

In [1]:
import os
import sqlite3
import json
import pandas as pd
os.getcwd()

'/Users/andrewtamez/Desktop/NPS/OA3802_Comp3/Final_Project'

First, we must establish a connection to the the SQL table we wish to create. Our data includes over 3 million data points with each table being 1 million points. We wil use 

`conn=sqlite3.connect('name_of_db.db')`

replace the name_of_db with the name you wish. 

We then create a cursor to allow use to query later in the code.

In [7]:
conn = sqlite3.connect('Football_stats.db')
cursor = conn.cursor()

Since we have our JSON files from our finalcalc code, we will import them so we can use the data to upload to our SQL table.

We will use:

`json_filename = 'filename.json`
`with open(json_filename, 'r') as f:`
    `filename.data =json.load(f)`

In our case here, we want to read the file. Ensure you are using `r` in the `with open`. 

In [4]:
json_superbowl = 'superbowl.json' 
with open(json_superbowl, 'r') as f:
    superbowl_data = json.load(f)
json_cc = 'ccwins.json' 
with open(json_cc, 'r') as f:
    cc_data = json.load(f)
json_playoffs = 'playoffs.json' 
with open(json_playoffs, 'r') as f:
    playoffs_data = json.load(f)

Now that we have our data, we want to create a table for each of the datasets. 

Using  `cursor.execute(f"CREATE TABLE IF NOT EXISTS Superbowl ({', '.join(columns)})")` allows us to create a table within the database. In our case, Superbowl is the name of the table but you are able to rename it to the table you wish. 

We then loop through our data to ensure we add it to our table via various assignments. 

Finally we use `cursor.execute(INSERT INTO table_name({', '.join(column_names)}) VALUES ({placeholders})", values)` that allows us to insert the data. We do this for each of the data sets. 


In [12]:
columns = ["QB", "TE", "DL", "P", "OL", "LB", "WR", "SEC", "K", "LS", 
    "RB", "predicted_probability"]

#Create table with the specified columns
cursor.execute(f"CREATE TABLE IF NOT EXISTS Superbowl ({', '.join(columns)})")
cursor.execute(f"CREATE TABLE IF NOT EXISTS CC ({', '.join(columns)})")
cursor.execute(f"CREATE TABLE IF NOT EXISTS Playoffs ({', '.join(columns)})")

#Insert data into the table with specified column names
for record in superbowl_data:
    spending = record['spending_percentages']
    predicted_probability = record['predicted_probability']
    
    # Prepare column names and values for insertion
    column_names = list(spending.keys()) + ['predicted_probability']
    values = list(spending.values()) + [predicted_probability]
    placeholders = ", ".join(["?"] * len(column_names))
    
    #Insert the record into the table with explicit column names
    cursor.execute(f"INSERT INTO Superbowl ({', '.join(column_names)}) VALUES ({placeholders})", values)

for record in cc_data:
    spending = record['spending_percentages']
    predicted_probability = record['predicted_probability']
    
    # Prepare column names and values for insertion
    column_names = list(spending.keys()) + ['predicted_probability']
    values = list(spending.values()) + [predicted_probability]
    placeholders = ", ".join(["?"] * len(column_names))
    
    # Insert the record into the table with explicit column names
    cursor.execute(f"INSERT INTO CC ({', '.join(column_names)}) VALUES ({placeholders})", values)

for record in playoffs_data:
    spending = record['spending_percentages']
    predicted_probability = record['predicted_probability']
    
    # Prepare column names and values for insertion
    column_names = list(spending.keys()) + ['predicted_probability']
    values = list(spending.values()) + [predicted_probability]
    placeholders = ", ".join(["?"] * len(column_names))
    
    # Insert the record into the table with explicit column names
    cursor.execute(f"INSERT INTO Playoffs ({', '.join(column_names)}) VALUES ({placeholders})", values)


This next step allows us to take the average of each column for specified probabilites in the 90th percentile determined when running our finalcalc code. 

We create a `query = SELECT AVG(column_name) FROM table_name WHERE condition`. 

We are taking the average from the specified table where the condition is met. 

We then run `cursor1.execute(query1)` to execute the action of the query using our cursor.  

In [ ]:
query1 = """
    SELECT 
        AVG(QB) AS avg_QB,
        AVG(TE) AS avg_TE,
        AVG(DL) AS avg_DL,
        AVG(P) AS avg_P,
        AVG(OL) AS avg_OL,
        AVG(LB) AS avg_LB,
        AVG(WR) AS avg_WR,
        AVG(SEC) AS avg_SEC,
        AVG(K) AS avg_K,
        AVG(LS) AS avg_LS,
        AVG(RB) AS avg_RB,
        AVG(predicted_probability) AS avg_predicted_probability
    FROM SuperBowl
    WHERE predicted_probability >= 0.56221 AND predicted_probability <= 0.59018
"""
query2 = """
    SELECT 
        AVG(QB) AS avg_QB,
        AVG(TE) AS avg_TE,
        AVG(DL) AS avg_DL,
        AVG(P) AS avg_P,
        AVG(OL) AS avg_OL,
        AVG(LB) AS avg_LB,
        AVG(WR) AS avg_WR,
        AVG(SEC) AS avg_SEC,
        AVG(K) AS avg_K,
        AVG(LS) AS avg_LS,
        AVG(RB) AS avg_RB,
        AVG(predicted_probability) AS avg_predicted_probability
    FROM CC
    WHERE predicted_probability >= 0.57028 AND predicted_probability <= 0.60272
"""
query3 = """
    SELECT 
        AVG(QB) AS avg_QB,
        AVG(TE) AS avg_TE,
        AVG(DL) AS avg_DL,
        AVG(P) AS avg_P,
        AVG(OL) AS avg_OL,
        AVG(LB) AS avg_LB,
        AVG(WR) AS avg_WR,
        AVG(SEC) AS avg_SEC,
        AVG(K) AS avg_K,
        AVG(LS) AS avg_LS,
        AVG(RB) AS avg_RB,
        AVG(predicted_probability) AS avg_predicted_probability
    FROM Playoffs
    WHERE predicted_probability >= 0.6188 AND predicted_probability <= 0.66913
"""
cursor.execute(query1)
cursor.execute(query2)
cursor.execute(query3)


The following code executes the query and we print out the values to ensure our averages were correctly taken. 

to execute the query use `cursor.execute(query)` to run your specified query. 

To get the results you utilize `cursor.fetchall()`. Ensure you have them separated as running the the query execute then running the print statement for the rows will result in the `fetchall()` pulling the resutls for the last query execute. Lesson learned on my end but you shouldn't have this issue if you take my advice. 

In [22]:
cursor.execute(query1)
print("Results from Query 1:")
results1 = cursor.fetchall()
for row in results1:
    print(row)

Results from Query 1:
(0.13036518515132742, 0.1270989267621871, 0.11956545238149745, 0.10357541443849798, 0.10103137619853778, 0.08412322522002538, 0.08360935660361205, 0.07519518774680445, 0.06023277990680225, 0.058658229534283284, 0.05654486605642487, 0.5654055130218425)


In [21]:
cursor.execute(query2)
print("\nResults from Query 2:")
results2 = cursor.fetchall()
for row in results2:
    print(row)


Results from Query 2:
(0.10663027714782307, 0.11207340576591435, 0.1466884791289462, 0.08202482492449814, 0.09449916287632551, 0.06043498905869588, 0.06959857892282299, 0.09524366257241837, 0.11034540500371282, 0.06209098039557304, 0.06037023420326965, 0.5736377870696535)


In [20]:
cursor.execute(query3)
print("\nResults from Query 3:")
results3 = cursor.fetchall()
for row in results3:
    print(row)



Results from Query 3:
(0.12533988355293924, 0.05854090591994142, 0.11963177842725703, 0.05850468689956796, 0.09141128114299166, 0.12627222844773142, 0.1017870662548892, 0.1098291616750712, 0.08137721823608958, 0.060024011079244065, 0.06728177836427728, 0.6250661862304845)


Finally, to ensure the data is saved you must use `conn.commit` otherwise the changes you made will not be saved. 

Ensure you close out your cursor and conn otherwise there may be issues with you or someone else trying to access the database using `cursor.close()` and `conn.close()`. 

In [23]:
conn.commit()
cursor.close()
conn.close()